In [1]:
!pip install --upgrade tables
!pip install eli5
!pip install xgboost


     |████████████████████████████████| 4.3MB 2.7MB/s 
  Found existing installation: tables 3.4.4
    Uninstalling tables-3.4.4:
      Successfully uninstalled tables-3.4.4
     |████████████████████████████████| 112kB 2.8MB/s 


In [2]:
import pandas as pd
import numpy as np

from sklearn.dummy import DummyRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

import xgboost as xgb

from sklearn.metrics import mean_absolute_error as mae
from sklearn.model_selection import cross_val_score, KFold

import eli5
from eli5.sklearn import PermutationImportance

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.metrics.scorer module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.feature_selection.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.feature_selection. Anything that cannot be imported from sklearn.feature_selection is now part of the private API.
  warnings.warn(message, FutureWarning)
Using TensorFlow backend.


In [3]:
cd "/content/drive/My Drive/Colab Notebooks/matrix/matrix_two/dw_matrix_car/"

/content/drive/My Drive/Colab Notebooks/matrix/matrix_two/dw_matrix_car


In [68]:
df = pd.read_hdf('data/car.h5')
df.shape


(106494, 155)

In [69]:
df['price_currency'].value_counts(normalize=True)*100

PLN    99.80844
EUR     0.19156
Name: price_currency, dtype: float64

In [0]:
SUFFIX_CAT = '__cat'
for feat in df.columns:
  if isinstance(df[feat][0], list): continue

  factorized_values = df[feat].factorize()[0]
  if SUFFIX_CAT in feat:
    df[feat] = factorized_values
  else:
    df[feat + SUFFIX_CAT] = factorized_values


In [71]:
cat_feats = [x for x in df.columns if SUFFIX_CAT in x]
cat_feats = [x for x in cat_feats if 'price' not in x]
cat_feats
len(cat_feats)

151

#DecisionTree

In [0]:
def run_model(model, feats):
  X = df[feats].values
  y = df['price_value'].values

  scores = cross_val_score(model, X, y, cv = 3, scoring='neg_mean_absolute_error')
  return np.mean(scores), np.std(scores)


In [24]:
run_model(DecisionTreeRegressor(max_depth=5), cat_feats)

(-19695.13091100928, 148.72570644015792)

#Random Forest

In [73]:
xgb_params = {
    'max_depth':5, 
    'n_estimators':50, 
    'learning_rate':0.1,
    'seed':0
}

run_model(xgb.XGBRegressor(**xgb_params), cat_feats)

[10:29:16] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[10:29:36] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[10:29:55] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-13108.379065811214, 74.32158265003798)

In [28]:
m = xgb.XGBRegressor(**xgb_params)
m.fit(X, y)

imp = PermutationImportance(m, random_state=0).fit(X, y)
0.1274 ± 0.0017 	param_napęd__cat
0.1159 ± 0.0025 	param_stan__cat
0.1131 ± 0.0037 	param_rok-produkcji__cat
0.0565 ± 0.0010 	param_faktura-vat__cat
0.0482 ± 0.0014 	param_moc__cat
0.0416 ± 0.0017 	param_skrzynia-biegów__cat
0.0252 ± 0.0010 	param_marka-pojazdu__cat
0.0247 ± 0.0014 	feature_kamera-cofania__cat
0.0232 ± 0.0008 	param_typ__cat
0.0198 ± 0.0006 	param_pojemność-skokowa__cat
0.0148 ± 0.0011 	seller_name__cat
0.0124 ± 0.0005 	param_wersja__cat
0.0124 ± 0.0009 	feature_wspomaganie-kierownicy__cat
0.0120 ± 0.0008 	param_model-pojazdu__cat
0.0113 ± 0.0003 	feature_system-start-stop__cat
0.0099 ± 0.0004 	param_kod-silnika__cat
0.0089 ± 0.0003 	feature_asystent-pasa-ruchu__cat
0.0081 ± 0.0004 	feature_łopatki-zmiany-biegów__cat
0.0076 ± 0.0002 	feature_światła-led__cat
0.0075 ± 0.0006 	feature_czujniki-parkowania-przednie__cat eli5.show_weights(imp, feature_names = cat_feats)

[09:45:16] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


Weight,Feature
0.1274 ± 0.0017,param_napęd__cat
0.1159 ± 0.0025,param_stan__cat
0.1131 ± 0.0037,param_rok-produkcji__cat
0.0565 ± 0.0010,param_faktura-vat__cat
0.0482 ± 0.0014,param_moc__cat
0.0416 ± 0.0017,param_skrzynia-biegów__cat
0.0252 ± 0.0010,param_marka-pojazdu__cat
0.0247 ± 0.0014,feature_kamera-cofania__cat
0.0232 ± 0.0008,param_typ__cat
0.0198 ± 0.0006,param_pojemność-skokowa__cat


In [74]:
feats = ['param_napęd__cat','param_stan__cat','param_rok-produkcji__cat','param_faktura-vat__cat','param_moc__cat','param_skrzynia-biegów__cat','param_marka-pojazdu__cat','feature_kamera-cofania__cat','param_typ__cat','param_pojemność-skokowa__cat','seller_name__cat','param_wersja__cat','feature_wspomaganie-kierownicy__cat','param_model-pojazdu__cat','feature_system-start-stop__cat','param_kod-silnika__cat','feature_asystent-pasa-ruchu__cat','feature_łopatki-zmiany-biegów__cat','feature_światła-led__cat','feature_czujniki-parkowania-przednie__cat']
run_model(xgb.XGBRegressor(**xgb_params), cat_feats)

[10:30:45] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[10:31:05] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[10:31:24] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-13108.379065811214, 74.32158265003798)

In [75]:
len(feats)

20

In [76]:
df['param_napęd'].unique()

array([None, 'Na przednie koła', '4x4 (dołączany automatycznie)',
       'Na tylne koła', '4x4 (dołączany ręcznie)', '4x4 (stały)'],
      dtype=object)

In [80]:
df['param_rok-produkcji'].unique()

array([2018, 2011, 2015, 2009, 2017, 2012, 2013, 2007, 2001, 2016, 2006,
       2008, 2004, 1999, 2000, 2010, 2005, 2002, 1998, 2014, 2003, 1982,
       1995, 1997, 1992, 1993, 1994, 1996, 1989, 1988, 1967, 1987, 1970,
       1959, 1990, 1991, 1974,   -1, 1975, 1973, 1953, 1985, 1984, 1986,
       1981, 1979, 1960, 1983, 1978, 1964, 1980, 1972, 1969, 1956, 1966,
       1977, 1962, 1965, 1971, 1963, 1961, 1952, 1949, 1976, 1937, 1968,
       1958, 1955, 1933, 1929, 1957, 1944, 1954, 1932, 1936, 1947, 1948])

In [78]:
df['param_rok-produkcji__cat'].unique()

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, -1, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49,
       50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66,
       67, 68, 69, 70, 71, 72, 73, 74, 75])

In [0]:
df['param_rok-produkcji'] = df['param_rok-produkcji'].map(lambda x: -1 if str(x) == 'None' else int(x))

In [47]:
feats = ['param_napęd__cat','param_stan__cat','param_rok-produkcji','param_faktura-vat__cat','param_moc__cat','param_skrzynia-biegów__cat','param_marka-pojazdu__cat','feature_kamera-cofania__cat','param_typ__cat','param_pojemność-skokowa__cat','seller_name__cat','param_wersja__cat','feature_wspomaganie-kierownicy__cat','param_model-pojazdu__cat','feature_system-start-stop__cat','param_kod-silnika__cat','feature_asystent-pasa-ruchu__cat','feature_łopatki-zmiany-biegów__cat','feature_światła-led__cat','feature_czujniki-parkowania-przednie__cat']
run_model(xgb.XGBRegressor(**xgb_params), feats)

[10:14:15] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[10:14:20] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[10:14:24] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-11386.828140541147, 58.212963494024464)

In [82]:
df['param_moc'] = df['param_moc'].map(lambda x:-1 if str(x) == 'None' else int(x.split(' ')[0]) )

df['param_moc'].unique()

array([ 90, 115, 262, 110, 310, 105, 140, 175, 125, 185, 190, 440, 141,
       200, 224,  75,  99, 184, 109, 233, 116,  68, 286, 126, 160, 135,
       120, 272,  -1, 150, 180, 136, 102, 131, 218, 245, 170, 112, 250,
       252,  73, 100, 313, 101, 285,  70, 383, 174, 277, 132, 130, 215,
        60, 330, 163, 177,  98,  78, 189, 156, 143,  69, 113,  65, 122,
        82, 251,  95, 197, 235, 238, 171, 381, 400, 178,  80, 165,  85,
       258, 142, 204, 124,  55, 144, 231, 248, 152, 181, 210, 340, 129,
       147,  50,  54, 290, 306, 193,  77, 164,  96, 194, 111, 166, 206,
       118, 360, 211, 271, 455, 280, 106, 114, 421,  74, 213, 121, 275,
       435, 384, 326,  88, 220, 260,  64,  86, 128, 256, 240, 244, 162,
       237, 350,  35, 265, 202, 133,  83, 117, 146,  92, 192, 145, 525,
       254, 182, 328, 367, 148, 456,  97, 270, 107, 108, 203, 155,  94,
        93, 241,  20,  71, 173,  58, 205, 236,   1, 557,  84, 457,  72,
       295, 134, 425, 228,  81, 230, 201,  87, 234, 299, 585, 20

In [50]:
feats = ['param_napęd__cat','param_stan__cat','param_rok-produkcji','param_faktura-vat__cat','param_moc','param_skrzynia-biegów__cat','param_marka-pojazdu__cat','feature_kamera-cofania__cat','param_typ__cat','param_pojemność-skokowa__cat','seller_name__cat','param_wersja__cat','feature_wspomaganie-kierownicy__cat','param_model-pojazdu__cat','feature_system-start-stop__cat','param_kod-silnika__cat','feature_asystent-pasa-ruchu__cat','feature_łopatki-zmiany-biegów__cat','feature_światła-led__cat','feature_czujniki-parkowania-przednie__cat']
run_model(xgb.XGBRegressor(**xgb_params), feats)

[10:20:23] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[10:20:27] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[10:20:31] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-9725.904859228242, 80.8769815926293)

In [90]:
df['param_pojemność-skokowa'] = df['param_pojemność-skokowa'].map(lambda x:-1 if str(x) == 'None' else int(str(x).split('cm')[0].replace(' ', '')) )
df['param_pojemność-skokowa'].unique()

array([   898,   1560,   3000, ...,   5992,   1966, 142280])

In [93]:
feats = ['param_napęd__cat','param_stan__cat','param_rok-produkcji','param_faktura-vat__cat','param_moc','param_skrzynia-biegów__cat','param_marka-pojazdu__cat','feature_kamera-cofania__cat','param_typ__cat','param_pojemność-skokowa','seller_name__cat','param_wersja__cat','feature_wspomaganie-kierownicy__cat','param_model-pojazdu__cat','feature_system-start-stop__cat','param_kod-silnika__cat','feature_asystent-pasa-ruchu__cat','feature_łopatki-zmiany-biegów__cat','feature_światła-led__cat','feature_czujniki-parkowania-przednie__cat']
run_model(xgb.XGBRegressor(**xgb_params), feats)

[10:57:32] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[10:57:35] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[10:57:39] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-9556.21383364917, 101.81252798368146)